# Beyond Metropolis: a more realistic example

As discussed in the previous notebook, the Metropolis algorithm is the basic building block of MCMC. It is *guaranteed* to converge on the target distribution, but convergence will be *more efficient* if the **jump** (or **proposal**) distribution is similar to the target. This motivates the development of **adaptive** algorithms, where the proposal distribution is "tuned" as the Markov chain progresses.

There are lots of ways of implemeting adaptation (as well as various other innovations for improving the sampling efficiency), but a detailed description of modern MCMC algorithms is beyond the scope of these notes. What's more, if you're a hydrologist or an ecologist (rather than a statistician), you're probably more interested in *applying* MCMC, rather than learning about all the mathematical details. Fortunately, if you understand the principles and motivation behind the Metropolis algorithm (see notebook 4), you should be in a good position to use some more sophisticated MCMC packages, without having to worry about the algorithms themselves.

## 1. Useful Python packages

In this notebook, we'll start off by introducing two Python packages that implement a variety of state-of-the-art MCMC algorithms. We'll then illustrate the *entire model development and calibration process* by building a very basic hydrological model from scratch, and then calibrating it using some real data and a very powerful, modern MCMC algorithm.

### 1.1. PyMC3

**[PyMC3]('https://github.com/pymc-devs/pymc3')** is a Python package offering a variety of sophisticated MCMC samplers, including **Hamiltonian Monte Carlo (HMC)** and the **No U-Turn Sampler (NUTS)**. It also provides a very clean syntax for model specification (defining priors, likelihoods etc.), as well as convenience functions for a wide range of distributions. This makes setting up a model and running MCMC very simple: as an illustration, check out the **linear regression** example in the [PyMC3 tutorial]('http://pymc-devs.github.io/pymc3/getting_started/'). This is very similar to the example covered in notebook 4, except there we coded everything from scratch and used only a basic Metropolis algorithm. With PyMC3, the number of lines of code is dramatically reduced and we have the added advantage of being able to switch easily between several modern samplers.

However, a key property of the HMC and NUTS algorithms is they use the **gradient** of the target function to help tune the proposal distribution. This gives excellent performance for high-dimensional problems, but requires the target function to be **differentiable**. Unfortunately, as we have already seen in notebook 3, for complex environmental models the likelihood function is often not well-behaved, so the posterior distribution is not differentiable and many of the advantages of the HMC and NUTS algorithms are lost.

### 1.2. emcee

**[emcee]('http://dan.iel.fm/emcee/current/')** is Python package which implements an **affine invariant MCMC ensemble sampler** (a more detailed description of the algorithm from the package authors is [here]('http://arxiv.org/abs/1202.3665')). This sampler uses **multiple chains** together with some clever mathematical "tricks" to tune the proposal distribution *without* requiring any gradient information. This makes it ideal for the kinds of complex likelihoods commonly encountered in environmental modelling. Furthermore, although in high-dimensional parameter spaces the HMC or NUTS algorithms *may* be more efficient, the multiple chains used by emcee are easy to parallelise (see notebook 4), which offers a significant performance advantage.

The downside of emcee is that, unlike PyMC3, there are no easy ways to specify your model: you have to write Python functions from scratch to represent your priors, likelihood and posterior. For simple models, this is not a major limitation, but for more complex likelihoods (e.g. more sophisticated error structures) the process can become time consuming.

[It is possible]('http://twiecki.github.io/blog/2013/09/23/emcee-pymc/') that the emcee algorithm will eventually be integrated into PyMC3, but in the meantime I recommend investigating both packages as they're useful in different situations. 

In the example below we'll use emcee.

## 2. A hydrological example

We're going to illustrate everything we've covered so far using a highly simpified but slightly more realisitc example. We'll formulate a very basic hydrological model to simulate flows in a small Scottish catchment. The model will be too simplistic for any real-world application, but I hope it will provide a useful illustration of the modelling process. We'll then use emcee to calibrate the model and see what we can learn about the parameter *given some real data*.

### 2.1. The study catchment

We're going to use some data from the **Tarland catchment** in Aberdeenshire, Scotland. Flow data are available from 2001 to 2010, together with rainfall and potential evapotranspiration (PET) datasets for the same period that we'll use to drive our model. 

**Tarland map here**

It is important to stress that *all* of these datasets are subject to **error**. For example:

  1. The rainfall data come from spatially interpolated rain gauge measurements. Rain gauges are notoriously inaccurate (especially in Scotland where it's often raining horizontally!) and the spatial interpolation process adds further uncertainty <br><br>
  
  2. The PET data has been estimated using the **[FAO56 modified Penman-Monteith method]('http://www.fao.org/docrep/x0490e/x0490e00.htm')**. This is a whole model in itself, incorporating data on cloud cover, solar radiation, humidity, wind speed, temperature etc., all of which are imperfectly measured. The method (as presented here) also assumes that the land cover is uniform grass of an even height, which is not the case in the Tarland catchment <br><br>
  
  3. The flow data are based on an empirically derived **[stage-discharge relationship]('http://water.usgs.gov/edu/streamflow3.html')** which is subject to considerable uncertainty due to changing bed sedimenst and limited gauging data, especially at high flows
  
Understanding these data limitations is clearly important and in a comprehensive analysis we might attempt to incorporate uncertainty about our input and calibration data into our likelihood function. Even if we do not do this, we should certainly spend some time **quality checking** the data and perhaps **cleaning or removing points that are obviously spurious**. For the simple example presented here we will *ignore all these issues* and instead focus only on the **parameter-related uncertainty** in our model. This is an important omission, but I want to keep this initial example as simple as possible.

The data have a daily time step and can be downloaded as follows:

In [ ]:
# Download Tarland data to Pandas df

### 2.2. A conceptual hydrological model

We are going to represent the entire Tarland catchment as **two connected "buckets"**, one representing the **soil water** reservoir and the other representing the **groundwater**. This approach is very common in hydrology, except most practically useful models break the system down into more buckets. For example, some models use different buckets for different combinations of land cover and soil type; others divide the subsurface into more than two reservoirs, for example by having separate "buckets" for shallow versus deep groundwater, and perhaps the same for soil water too. There are many variations on this theme, and once you have many buckets you find there are numerous ways of linking them together, which leads to consideration of e.g. water travel times along different river reaches and between different water stores. 

All of this adds to the complexity of the model and can quickly lead to **over-parameterisation**, as described in notebook 2. We're going to *ignore all of these complexities as well* and just consider two simple buckets. With such a simple setup we certainly can't be accused of over-parameterisation, but equally it's unlikely that this model will be very informative in terms of understanding the hydrology of the Tarland catchment.

**Image of two-bucket model here**

As part of our conceptualisation, we will assume that the amount of water, $R$, entering the soil bucket at each time step is equal to the precipitation, $P$, minus the PET, $E$

$$R = P - E$$

Throughout the entire analysis, all water **volumes** will be expressed in **millimetres**. This is not a mistake - we're simply performing all of the calculation **per unit area** so that we can visualise what's going on in terms of the 2D sketch above. In reality the Tarland catchment has an area of about 50 $km^2$, so if $R$, the incoming **hydrologically effective rainfall**, is 1 mm, this equates to an actual water volume of $1.10^{-3} * 50.10^6 = 50.10^3$ $m^3$. 

Water is assume to drain from the soil bucket at 